In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def prepare_temporal_data(df):
    """Prepara datos desde 2020-12-01"""
    temporal_df = df.copy()
    temporal_df['date_received'] = pd.to_datetime(temporal_df['date_received'])
    
    # Filtrar desde 2020-12-01
    start_date = pd.to_datetime('2020-12-01')
    temporal_df = temporal_df[temporal_df['date_received'] >= start_date]
    
    temporal_df['quarter'] = temporal_df['date_received'].dt.to_period('Q')
    temporal_df['year_month'] = temporal_df['date_received'].dt.to_period('M')
    
    print(f"Datos filtrados: {len(temporal_df):,} registros")
    print(f"Productos únicos: {temporal_df['product'].nunique()}")
    
    return temporal_df

def analyze_quarterly_evolution(df):
    """Análisis trimestral optimizado"""
    # Top 6 productos por volumen total
    top_products = df['product'].value_counts().head(6).index
    
    # Evolución trimestral
    quarterly_data = df[df['product'].isin(top_products)].groupby(['quarter', 'product']).size().unstack(fill_value=0)
    
    plt.figure(figsize=(14, 8))
    for product in quarterly_data.columns:
        plt.plot(quarterly_data.index.astype(str), quarterly_data[product], marker='o', linewidth=2, label=product[:30])
    
    plt.title('Evolución Trimestral - Top 6 Productos (2020-2025)', fontsize=14, fontweight='bold')
    plt.xlabel('Trimestre')
    plt.ylabel('Número de Quejas')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return quarterly_data

def analyze_yearly_heatmap(df):
    """Heatmap anual simplificado"""
    # Top 10 productos para mejor visualización
    top_products = df['product'].value_counts().head(10).index
    
    yearly_data = df[df['product'].isin(top_products)].groupby(['year', 'product']).size().unstack(fill_value=0)
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(yearly_data.T, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Número de Quejas'})
    plt.title('Heatmap Anual - Top 10 Productos (2021-2025)', fontsize=14, fontweight='bold')
    plt.xlabel('Año')
    plt.ylabel('Producto')
    plt.tight_layout()
    plt.show()
    
    return yearly_data

def detect_emerging_declining_products(df):
    """Detecta productos emergentes y en declive"""
    # Agrupar por año y producto
    yearly_counts = df.groupby(['year', 'product']).size().unstack(fill_value=0)
    
    # Filtrar productos con al menos 50 quejas totales (ajustado para el dataset filtrado)
    product_totals = yearly_counts.sum(axis=0)
    relevant_products = product_totals[product_totals >= 50].index
    yearly_counts = yearly_counts[relevant_products]
    
    # Calcular cambio entre primer y último año disponible
    first_year = yearly_counts.index.min()
    last_year = yearly_counts.index.max()
    
    changes = {}
    for product in yearly_counts.columns:
        first_val = yearly_counts.loc[first_year, product] if first_year in yearly_counts.index else 0
        last_val = yearly_counts.loc[last_year, product] if last_year in yearly_counts.index else 0
        
        if first_val > 0:
            change_pct = ((last_val - first_val) / first_val) * 100
            changes[product] = change_pct
    
    # Productos emergentes (>30% crecimiento) y en declive (>20% decrecimiento)
    emerging = {k: v for k, v in changes.items() if v > 30}
    declining = {k: v for k, v in changes.items() if v < -20}
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    if emerging:
        products_em = list(emerging.keys())[:5]  # Top 5
        values_em = [emerging[p] for p in products_em]
        ax1.barh(range(len(products_em)), values_em, color='green', alpha=0.7)
        ax1.set_yticks(range(len(products_em)))
        ax1.set_yticklabels([p[:25] + '...' if len(p) > 25 else p for p in products_em])
        ax1.set_title('Productos Emergentes\n(Crecimiento >30%)')
        ax1.set_xlabel('% Cambio')
    
    if declining:
        products_dec = list(declining.keys())[:5]  # Top 5
        values_dec = [declining[p] for p in products_dec]
        ax2.barh(range(len(products_dec)), values_dec, color='red', alpha=0.7)
        ax2.set_yticks(range(len(products_dec)))
        ax2.set_yticklabels([p[:25] + '...' if len(p) > 25 else p for p in products_dec])
        ax2.set_title('Productos en Declive\n(Decrecimiento >20%)')
        ax2.set_xlabel('% Cambio')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Productos emergentes encontrados: {len(emerging)}")
    print(f"Productos en declive encontrados: {len(declining)}")
    
    return emerging, declining

def analyze_monthly_volatility(df):
    """Análisis de volatilidad mensual para top 3 productos"""
    top_products = df['product'].value_counts().head(3).index
    
    monthly_data = df[df['product'].isin(top_products)].groupby(['year_month', 'product']).size().unstack(fill_value=0)
    
    # Calcular coeficiente de variación
    volatility = {}
    for product in monthly_data.columns:
        cv = monthly_data[product].std() / monthly_data[product].mean() if monthly_data[product].mean() > 0 else 0
        volatility[product] = cv
    
    plt.figure(figsize=(14, 6))
    for product in monthly_data.columns:
        plt.plot(monthly_data.index.astype(str), monthly_data[product], marker='o', alpha=0.7, label=product[:30])
    
    plt.title('Evolución Mensual - Top 3 Productos (2020-2025)', fontsize=14, fontweight='bold')
    plt.xlabel('Mes')
    plt.ylabel('Número de Quejas')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("Coeficiente de Variación (Volatilidad):")
    for product, cv in volatility.items():
        print(f"{product[:40]}: {cv:.3f}")
    
    return monthly_data, volatility

def run_temporal_analysis(new_df):
    """Ejecuta análisis temporal completo optimizado"""
    print("🔍 ANÁLISIS TEMPORAL AVANZADO (2020-2025)")
    print("=" * 50)
    
    # Preparar datos
    temporal_df = prepare_temporal_data(new_df)
    
    print("\n📊 1. Evolución Trimestral")
    quarterly_data = analyze_quarterly_evolution(temporal_df)
    
    print("\n🔥 2. Heatmap Anual")
    yearly_data = analyze_yearly_heatmap(temporal_df)
    
    print("\n📈 3. Productos Emergentes y en Declive")
    emerging, declining = detect_emerging_declining_products(temporal_df)
    
    print("\n📉 4. Análisis de Volatilidad Mensual")
    monthly_data, volatility = analyze_monthly_volatility(temporal_df)
    
    print("\n✅ Análisis temporal completado!")
    
    return {
        'temporal_df': temporal_df,
        'quarterly_data': quarterly_data,
        'yearly_data': yearly_data,
        'emerging_products': emerging,
        'declining_products': declining,
        'monthly_data': monthly_data,
        'volatility': volatility
    }